In [4]:
import numpy as np
import pandas as pd
import time

# 1. Chargement du fichier CSV réel
try:
    df = pd.read_csv('NBA_Data.csv')
    print("Fichier chargé avec succès.")
    print(f"Dimensions originales : {df.shape}")
except FileNotFoundError:
    print("ERREUR : Le fichier 'NBA_Data.csv' est introuvable au même niveau que le notebook.")

# 2. Sélection des colonnes numériques pour l'exercice
# On ne garde que les colonnes utiles pour les calculs (Points, Rebonds, Passes)
colonnes_utiles = ['PTS', 'REB', 'AST'] 

# Vérification que les colonnes existent
if set(colonnes_utiles).issubset(df.columns):
    df_stats = df[colonnes_utiles].copy()
    
    # 3. Nettoyage : Remplacer les valeurs manquantes (NaN) par 0
    # C'est crucial avec les vraies données, sinon le résultat sera 'nan'
    df_stats = df_stats.fillna(0)
    
    # 4. Conversion en NumPy array
    arr = df_stats.to_numpy()
    
    print(f"\nDonnées numériques prêtes pour le calcul.")
    print(f"Shape du tableau numpy : {arr.shape}")
    print("\nAperçu des 5 premières lignes (PTS, REB, AST) :")
    print(arr[:5])
else:
    print(f"\nERREUR : Votre CSV ne contient pas exactement les colonnes {colonnes_utiles}.")
    print(f"Colonnes disponibles : {list(df.columns)}")

Fichier chargé avec succès.
Dimensions originales : (9130, 24)

Données numériques prêtes pour le calcul.
Shape du tableau numpy : (9130, 3)

Aperçu des 5 premières lignes (PTS, REB, AST) :
[[33.9  9.2  9.8]
 [30.4 11.5  6.5]
 [30.1  5.5  6.2]
 [28.7  3.6  6.7]
 [27.1  6.6  5. ]]


In [15]:
print("--- Comparaison sur données réelles ---")

if 'arr' in locals():
    # 1. Somme avec np.sum
    start = time.time()
    total_sum = np.sum(arr, axis=0)
    end = time.time()
    temps_sum = end - start
    
    print(f"Total (PTS, REB, AST) avec np.sum :")
    print(total_sum)
    print(f"Temps : {temps_sum:.20f} s")

    # 2. Somme avec np.einsum
    start = time.time()
    # 'ij->j' : Somme sur l'axe i (les joueurs), garde l'axe j (les 3 stats)
    total_einsum = np.einsum('ij->j', arr)
    end = time.time()
    temps_einsum = end - start
    
    print(f"\nTotal (PTS, REB, AST) avec np.einsum :")
    print(total_einsum)
    print(f"Temps : {temps_einsum:.20f} s")
    
    # Vérification
    print(f"\nRésultats identiques ? {np.allclose(total_sum, total_einsum)}")
else:
    print("Veuillez exécuter la Cellule 1 d'abord.")

--- Comparaison sur données réelles ---
Total (PTS, REB, AST) avec np.sum :
[82852.9 34923.3 18150.9]
Temps : 0.00251293182373046875 s

Total (PTS, REB, AST) avec np.einsum :
[82852.9 34923.3 18150.9]
Temps : 0.00000000000000000000 s

Résultats identiques ? True


In [29]:
print("--- Calcul des scores joueurs (Somme pondérée) ---")

if 'arr' in locals():
    # Poids arbitraires : 1 pt = 0.5, 1 reb = 1.5, 1 ast = 2.0
    weights = np.array([0.5, 1.5, 2.0]) 

    # Méthode Einsum (souvent plus efficace en mémoire)
    start = time.time()
    # 'ij,j->i' : Pour chaque joueur (i), on multiplie ses stats (j) par les poids (j) et on somme
    scores_joueurs = np.einsum('ij,j->i', arr, weights)
    end = time.time()
    
    print(f"Temps de calcul : {end - start:.10f} s")
    
    # --- Bonus : Afficher les meilleurs joueurs ---
    # On remet le score dans le DataFrame pour voir qui est qui
    df['Custom_Score'] = scores_joueurs
    
    # Si la colonne 'player' ou 'Player' existe, on affiche le top 5
    col_nom = 'player' if 'player' in df.columns else 'Player'
    
    if col_nom in df.columns:
        print(f"\nTop 5 des joueurs selon notre score pondéré :")
        print(df[[col_nom, 'PTS', 'REB', 'AST', 'Custom_Score']]
              .sort_values(by='Custom_Score', ascending=False)
              .head(5))
    else:
        print("\nScores calculés (5 premiers) :", scores_joueurs[:5])

--- Calcul des scores joueurs (Somme pondérée) ---
Temps de calcul : 0.0010118484 s

Top 5 des joueurs selon notre score pondéré :
                 player   PTS   REB   AST  Custom_Score
3095  Russell Westbrook  31.6  10.7  10.4         52.65
1397  Russell Westbrook  22.2  11.5  11.7         51.75
0           Luka Doncic  33.9   9.2   9.8         50.35
912        Nikola Jokic  27.1  13.8   7.9         50.05
9          Nikola Jokic  26.4  12.4   9.0         49.80
